In [ ]:
import pygame
import sys
import csv
import os
import math
from datetime import datetime
import random

# -------------------------
# PARAMÈTRES
# -------------------------

SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 700

REF_BASE_LENGTH = 260          # longueur de base (px)
LENGTH_DIFFS = [-40, -20, 0, 20, 40]  # top_length - bottom_length
TRIALS_PER_DIFF = 10           # nb d'essais par différence

TOP_Y = SCREEN_HEIGHT // 2 - 80
BOTTOM_Y = SCREEN_HEIGHT // 2 + 80

FIXATION_TIME = 500            # ms

# -------------------------
# FONCTIONS D'AFFICHAGE
# -------------------------

def draw_ponzo_frame(screen):
    """Dessine deux lignes convergentes (cadre de Ponzo)."""
    color = (200, 200, 200)
    # lignes convergentes comme une route en perspective
    pygame.draw.line(screen, color,
                     (SCREEN_WIDTH // 2 - 250, SCREEN_HEIGHT),
                     (SCREEN_WIDTH // 2 - 40, 150), 3)
    pygame.draw.line(screen, color,
                     (SCREEN_WIDTH // 2 + 250, SCREEN_HEIGHT),
                     (SCREEN_WIDTH // 2 + 40, 150), 3)

def draw_bar(screen, length, y, color=(255, 255, 255), thickness=4):
    """Dessine une barre horizontale centrée."""
    x1 = SCREEN_WIDTH // 2 - length // 2
    x2 = SCREEN_WIDTH // 2 + length // 2
    pygame.draw.line(screen, color, (x1, y), (x2, y), thickness)

def show_text_center(screen, text, size=32):
    font = pygame.font.Font(None, size)
    lines = text.split("\n")
    total_height = len(lines) * size * 1.4
    start_y = SCREEN_HEIGHT // 2 - total_height // 2

    for i, line in enumerate(lines):
        surf = font.render(line, True, (255, 255, 255))
        rect = surf.get_rect(center=(SCREEN_WIDTH // 2,
                                     start_y + i * size * 1.4))
        screen.blit(surf, rect)

# -------------------------
# EXPÉRIENCE PRINCIPALE
# -------------------------

def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Illusion de Ponzo")
    clock = pygame.time.Clock()

    # ----- Identifiant sujet (console)
    subject_id = input("Entrez l'identifiant sujet (ex: S01): ")

    # ----- Préparation fichier CSV
    if not os.path.exists("data"):
        os.makedirs("data")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/{subject_id}_ponzo_{timestamp}.csv"

    fieldnames = [
        "subject_id", "trial_index",
        "length_diff", "top_length", "bottom_length",
        "correct_answer", "response", "is_correct", "rt"
    ]

    csvfile = open(filename, "w", newline="", encoding="utf-8")
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # ----- Instructions
    screen.fill((0, 0, 0))
    show_text_center(
        screen,
        "Illusion de Ponzo\n\n"
        "Deux lignes sont présentées entre des lignes convergentes.\n"
        "Quelle ligne PARAÎT la plus longue ?\n\n"
        "Flèche HAUT = ligne du haut plus longue\n"
        "Flèche BAS  = ligne du bas plus longue\n\n"
        "Appuyez sur ESPACE pour commencer.",
        size=28
    )
    pygame.display.flip()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                csvfile.close()
                pygame.quit()
                sys.exit(0)
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    waiting = False

    # ----- Génération des essais
    trials = []
    for diff in LENGTH_DIFFS:
        for _ in range(TRIALS_PER_DIFF):
            trials.append(diff)
    random.shuffle(trials)

    trial_index = 0

    # ----- Boucle expérimentale
    for length_diff in trials:
        trial_index += 1

        # Définir les longueurs physiques
        bottom_length = REF_BASE_LENGTH
        top_length = REF_BASE_LENGTH + length_diff

        # Déterminer la bonne réponse
        if top_length > bottom_length:
            correct_answer = "top"
        elif top_length < bottom_length:
            correct_answer = "bottom"
        else:
            correct_answer = "equal"  # cas longueur égale

        # Fixation
        screen.fill((0, 0, 0))
        show_text_center(screen, "+", size=80)
        pygame.display.flip()
        pygame.time.wait(FIXATION_TIME)

        # Début de l'essai
        rt_start = pygame.time.get_ticks()
        response = None
        is_correct = None

        running_trial = True
        while running_trial:
            screen.fill((0, 0, 0))

            # Cadre de Ponzo
            draw_ponzo_frame(screen)

            # Barre du haut
            draw_bar(screen, top_length, TOP_Y)

            # Barre du bas
            draw_bar(screen, bottom_length, BOTTOM_Y)

            pygame.display.flip()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    csvfile.close()
                    pygame.quit()
                    sys.exit(0)
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        csvfile.close()
                        pygame.quit()
                        sys.exit(0)
                    # Réponse : flèche haut ou bas
                    if event.key == pygame.K_UP:
                        response = "top"
                        running_trial = False
                    if event.key == pygame.K_DOWN:
                        response = "bottom"
                        running_trial = False

            clock.tick(60)

        rt = (pygame.time.get_ticks() - rt_start) / 1000.0

        # Déterminer justesse
        if correct_answer == "equal":
            # si longueurs égales, on ne définit pas "correct / incorrect"
            is_correct = ""
        else:
            is_correct = (response == correct_answer)

        # Sauvegarde essai
        writer.writerow({
            "subject_id": subject_id,
            "trial_index": trial_index,
            "length_diff": length_diff,
            "top_length": top_length,
            "bottom_length": bottom_length,
            "correct_answer": correct_answer,
            "response": response,
            "is_correct": is_correct,
            "rt": rt
        })
        csvfile.flush()

    # ----- Fin
    screen.fill((0, 0, 0))
    show_text_center(screen, "Merci de votre participation !", size=40)
    pygame.display.flip()
    pygame.time.wait(2000)

    csvfile.close()
    pygame.quit()
    print(f"Données sauvegardées dans : {filename}")

if __name__ == "__main__":
    main()
